# Import statement

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
from operator import itemgetter

#Natural language toolkit
import nltk
from nltk.stem import PorterStemmer

#libraries to build vectors with dimensions which are every single words
from sklearn.feature_extraction.text import  TfidfVectorizer, CountVectorizer

#Cosin library to calculate the angle of every two vector
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

import pickle


# 1. Loading data and EDA

In [2]:
path = '../../Data'
movies = pd.read_csv(path + '/movies_metadata.csv')
credits = pd.read_csv(path + '/credits.csv')
keywords = pd.read_csv(path + '/keywords.csv')
ratings = pd.read_csv(path + '/ratings_small.csv')
links_small = pd.read_csv(path + '/links_small.csv')

C:\Users\baomi\AppData\Local\Temp\ipykernel_14836\3992106928.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(path + '/movies_metadata.csv')


In [3]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30-10-95,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15-12-95,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22-12-95,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22-12-95,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10-02-95,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
movies.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [5]:
links_small.tmdbId = links_small[links_small['tmdbId'].notnull()].tmdbId.astype('int')
links_small.rename(columns={"tmdbId": "id"}, inplace=True)

In [6]:
movies = movies.drop([19730, 29503, 35587])

In [7]:
# Check EDA Notebook for how and why I got these indices.
movies.id = movies.id.astype('int')

In [8]:
smovies = links_small.merge(movies, on = "id")
smovies.shape

(9099, 26)

In [9]:
# links_small.id = links_small.id.astype('int')
credits.id = credits.id.astype('int')
keywords.id = keywords.id.astype('int')
smovies = smovies.merge(credits,on = 'id',how='left').merge(keywords, on = 'id', how='left')
smovies.shape

(9219, 29)

# 2. Preprocessing data

In [10]:
smovies = smovies[['id','title','genres','overview','tagline','cast','crew','keywords','production_companies','belongs_to_collection']]

In [11]:
smovies.isnull().sum()

id                          0
title                       0
genres                      0
overview                   12
tagline                  2137
cast                        0
crew                        0
keywords                    0
production_companies        0
belongs_to_collection    7531
dtype: int64

In [12]:
smovies['tagline'] = smovies['tagline'].fillna('')
smovies['overview'] = smovies['overview'].fillna('')
smovies['belongs_to_collection'] = smovies['belongs_to_collection'].fillna('{}')

In [13]:
smovies.cast[0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [14]:
smovies.keywords[0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [15]:
smovies.belongs_to_collection[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [16]:
literal_eval(smovies.belongs_to_collection[1]).get('name') or ''

''

In [17]:
def convert(text):
    list = []
    for i in literal_eval(text):
        list.append(i['name'])
    return list

In [18]:
smovies.genres = smovies.genres.apply(convert)
smovies.production_companies = smovies.production_companies.apply(convert)
smovies.cast = smovies.cast.apply(convert)
smovies.keywords = smovies.keywords.apply(convert)
smovies.belongs_to_collection = smovies.belongs_to_collection.apply(lambda x:literal_eval(x).get('name') or '')

In [19]:
smovies.head(1)

,id,title,genres,overview,tagline,cast,crew,keywords,production_companies,belongs_to_collection
0,862.0,Toy Story,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",[Pixar Animation Studios],Toy Story Collection


In [20]:
smovies.crew[0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [21]:
def convert_crew(text):
    list = []
    for i in literal_eval(text):
        list.append(itemgetter('job','name')(i))
    mytuple = dict((x, y) for x, y in list)
    return mytuple
convert_crew(smovies.crew[0])

{'Director': 'John Lasseter',
 'Screenplay': 'Alec Sokolow',
 'Producer': 'Ralph Guggenheim',
 'Executive Producer': 'Steve Jobs',
 'Editor': 'Robert Gordon',
 'Art Direction': 'Ralph Eggleston',
 'Foley Editor': 'Mary Helen Leasman',
 'Animation': 'Ken Willard',
 'ADR Editor': 'Marilyn McCoppen',
 'Orchestrator': 'Don Davis',
 'Color Timer': 'Dale E. Grahn',
 'CG Painter': 'William Cone',
 'Original Story': 'Andrew Stanton',
 'Post Production Supervisor': 'Patsy Bouge',
 'Sculptor': 'Shelley Daniels Lekven',
 'Animation Director': 'Rich Quade',
 'Music': 'Randy Newman',
 'Layout': 'Desirée Mourad',
 'Music Editor': 'James Flamberg',
 'Negative Cutter': 'Rick Mackay',
 'Title Designer': 'Susan Bradley',
 'Supervising Technical Director': 'William Reeves',
 'Songs': 'Randy Newman',
 'Supervising Animator': 'Pete Docter',
 'Sound Designer': 'Gary Rydstrom',
 'Production Supervisor': 'Karen Robert Jackson',
 'Executive Music Producer': 'Chris Montan',
 'Visual Effects Supervisor': 'Thomas

In [22]:
smovies.crew = smovies.crew.apply(convert_crew)
smovies.crew = smovies.crew.apply(lambda x: x.get('Director') or '')

In [23]:
smovies.head()

,id,title,genres,overview,tagline,cast,crew,keywords,production_companies,belongs_to_collection
0,862.0,Toy Story,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,"[jealousy, toy, boy, friendship, friends, riva...",[Pixar Animation Studios],Toy Story Collection
1,8844.0,Jumanji,"[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston,"[board game, disappearance, based on children'...","[TriStar Pictures, Teitler Film, Interscope Co...",
2,15602.0,Grumpier Old Men,"[Romance, Comedy]",A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch,"[fishing, best friend, duringcreditsstinger, o...","[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,31357.0,Waiting to Exhale,"[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker,"[based on novel, interracial relationship, sin...",[Twentieth Century Fox Film Corporation],
4,11862.0,Father of the Bride Part II,[Comedy],Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer,"[baby, midlife crisis, confidence, aging, daug...","[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection


In [24]:
def remove_space(words):
    list = []
    for i in words:
        list.append(i.replace(' ',''))
    return list

In [25]:
# smovies.crew.fillna('',inplace = True)

In [26]:
smovies.genres = smovies.genres.apply(remove_space)
smovies.cast = smovies.cast.apply(remove_space)
smovies.crew = smovies.crew.apply(lambda x: x.replace(' ',''))
smovies.keywords = smovies.keywords.apply(remove_space)
smovies.belongs_to_collection = smovies.belongs_to_collection.apply(lambda x: x.replace(' ',''))
smovies.production_companies = smovies.production_companies.apply(remove_space)

In [27]:
smovies.genres = smovies.genres.apply(' '.join)
smovies.cast = smovies.cast.apply(' '.join)
smovies.keywords = smovies.keywords.apply(' '.join)
smovies.production_companies = smovies.production_companies.apply(' '.join)

In [28]:
smovies['tags'] = smovies.genres + ' ' + smovies.overview + ' ' + smovies.cast + ' ' + smovies.crew +' ' + smovies.keywords + ' ' + (smovies.production_companies + ' ')*3 + ' ' + (smovies.belongs_to_collection + ' ')*4

In [29]:
smovies = smovies[['id','title','tags']]
smovies.tags = smovies.tags.apply(lambda x:x.lower())

In [30]:
smovies.tags[0]

"animation comedy family led by woody, andy's toys live happily in his room until andy's birthday brings buzz lightyear onto the scene. afraid of losing his place in andy's heart, woody plots against buzz. but when circumstances separate buzz and woody from their owner, the duo eventually learns to put aside their differences. tomhanks timallen donrickles jimvarney wallaceshawn johnratzenberger anniepotts johnmorris erikvondetten lauriemetcalf r.leeermey sarahfreeman pennjillette johnlasseter jealousy toy boy friendship friends rivalry boynextdoor newtoy toycomestolife pixaranimationstudios pixaranimationstudios pixaranimationstudios  toystorycollection toystorycollection toystorycollection toystorycollection "

# 3. Implement recommender systems

## 3.1 Content based system (NLP)

In [31]:
ps = PorterStemmer()

In [32]:
# format the some word to be harmonized such as "goes" -> "go"
def stems(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)

In [33]:
smovies.tags = smovies.tags.apply(stems)

In [34]:
smovies.tags[0]

"anim comedi famili led by woody, andy' toy live happili in hi room until andy' birthday bring buzz lightyear onto the scene. afraid of lose hi place in andy' heart, woodi plot against buzz. but when circumst separ buzz and woodi from their owner, the duo eventu learn to put asid their differences. tomhank timallen donrickl jimvarney wallaceshawn johnratzenberg anniepott johnmorri erikvondetten lauriemetcalf r.leeermey sarahfreeman pennjillett johnlasset jealousi toy boy friendship friend rivalri boynextdoor newtoy toycomestolif pixaranimationstudio pixaranimationstudio pixaranimationstudio toystorycollect toystorycollect toystorycollect toystorycollect"

In [35]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=1, stop_words='english')
count_matrix = count.fit_transform(smovies['tags'])

In [36]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [37]:
indices = smovies[['id','title']]
def get_recommendations(title):
    idx = indices[indices.title == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:10]
    movie_indices = [i[0] for i in sim_scores]
    return indices.iloc[movie_indices].title

In [38]:
get_recommendations('Avengers: Age of Ultron')

8868                Avengers: Age of Ultron
7953                           The Avengers
8618    Captain America: The Winter Soldier
7853                                   Thor
7592                             Iron Man 2
8712                Guardians of the Galaxy
8505                   Thor: The Dark World
8368                             Iron Man 3
8872             Captain America: Civil War
7001                               Iron Man
Name: title, dtype: object

In [39]:
get_recommendations('Spider-Man').head(10)

4111                  Spider-Man
5530                Spider-Man 2
6749                Spider-Man 3
8628    The Amazing Spider-Man 2
8150      The Amazing Spider-Man
2176               Arachnophobia
3932                    Blankman
8869                     Ant-Man
2227                        Dick
346                         Wolf
Name: title, dtype: object

In [40]:
pickle.dump(indices,open("../Arts/indices.pkl","wb"))
pickle.dump(cosine_sim,open("../../cosine_sim.pkl","wb"))

In [41]:
#Test git
indices.head()

,id,title
0,862.0,Toy Story
1,8844.0,Jumanji
2,15602.0,Grumpier Old Men
3,31357.0,Waiting to Exhale
4,11862.0,Father of the Bride Part II
